# Measurement of an Impulse Response with a Microphone

This notebook sketches how to measure impulse responses with a single microphone.

* Two sweeps are played, one for the loudspeaker and one for the analog feedback. 
* Two channels are recorded, the microphone and the analog feedback.

In [ ]:
# Automatically reload the acoustics_hardware package if anything is changed
%load_ext autoreload
%autoreload 2

# Load requirements
import sys

import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from IPython import display
from numpy.fft import rfft as fft, rfftfreq as fftfreq
from pathlib import Path
from time import sleep

import acoustics_hardware.serial

## Set up audio hardware

In [ ]:
# Show list of available audio devices
print(acoustics_hardware.devices.AudioDevice.get_devices())

# Audio device configuration
device_name = 'Orion 32'
fs = 48000 # in Hz

print(f'\nConnect to audio device "{device_name}" ... ', end='')
try:
    device = acoustics_hardware.devices.AudioDevice(
        name=device_name,
        fs=fs, # in Hz
    )
except ValueError as e:
    sys.exit(e)
print('Done.')

# 2 inputs (0..31) for 'Orion 32'
device.add_input(0) # add feedback output channel first
device.add_input(1)

# 2 outputs (0..31) for 'Orion 32'
device.add_output(0) # add feedback input channel first
device.add_output(1)

# Gather input and output channels
ch_inputs = [ch.index for ch in device.inputs]
ch_outputs = [ch.index for ch in device.outputs]
print(f'\nConnected input channels:  {ch_inputs}.')
print(f'Connected output channels: {ch_outputs}.')

# Sweep configuration
sweep_amplitude = -20 # in dB_FS
sweep_duration = 3.0 # in s
sweep_hp, sweep_lp = 50, 22e3 # in Hz

# Signal is automatically played through the first 2 channels
print(f'\nGenerate sweep ... ', end='')
generator = acoustics_hardware.generators.SweepGenerator(
    device=device,
    start_frequency=sweep_hp, # in Hz
    stop_frequency=sweep_lp, # in Hz
    duration=sweep_duration, # in s
    method='logarithmic',
    bidirectional=False,
    repetitions=1, # number of sweeps
    amplitude=10**(sweep_amplitude/20), # as factor
)
print('Done.')

## Prepare functions for plotting

In [ ]:
# noinspection PyShadowingNames
def plot_data(data, fs, ch_labels=None, is_stacked=False):
    data = np.atleast_2d(data)
    n_ch, n_sample = data.shape
    if ch_labels is not None and len(ch_labels) != n_ch:
        sys.exit(f'Mismatch in number of audio channels ({n_ch}) '
                 f'and number of provided labels {len(ch_labels)}.')
        
    t_data = np.linspace(0, n_sample / fs, n_sample)
    freq_data = 20*np.log10(abs(fft(data)))
    f = fftfreq(n_sample, 1 / fs)
    
    if is_stacked: n_ch = 1
        
    fig, axes = plt.subplots(
        nrows=n_ch,
        ncols=2 if is_stacked else 3,
        squeeze=False,
        sharex='col',
        figsize=(15, 5 if is_stacked else(3 * n_ch))
    )
    for ch in np.arange(n_ch): # individual channels
        ch_plot = (range(data.shape[0]) if is_stacked
                   else ch)
        
        # Time domain
        ax = axes[ch, 0]
        ax.plot(t_data, data[ch_plot, :].T)
        ax.set_xlim(0, n_sample / fs) # in s
        ax.grid()
        if ch_labels is not None:
            label = ([f'in_{l}' for l in ch_labels] if is_stacked 
                     else [f'in_{ch_labels[ch]}'])
            ax.legend(label, loc='upper right')
        ax.set_xlabel('Time (s)' if ch >= n_ch - 1 else '')
        ax.set_ylabel('Amplitude')

        # Frequency domain
        y_max = np.ceil(freq_data[ch_plot, :].max() / 5) * 5
        ax = axes[ch, 1]
        ax.semilogx(f, freq_data[ch_plot, :].T)
        ax.set_xlim(20, fs/2) # in Hz
        ax.set_ylim(y_max - (120 if is_stacked else 80), y_max) # in dB
        ax.grid()
        ax.set_xlabel('Frequency (Hz)' if ch >= n_ch - 1 else '')
        ax.set_ylabel('Magnitude (dB)')
        
        if not is_stacked:
            # Spectrogram
            ax = axes[ch, 2]
            ax.specgram(
                x=data[ch_plot, :].T,
                Fs=fs, # in Hz
                NFFT=1024, # in samples
                noverlap=512, # in samples
                mode='magnitude',
                scale='dB',
            )
            ax.set_yscale('log')
            ax.set_ylim(100, fs/2) # in Hz
            ax.set_xlabel('Time (s)' if ch >= n_ch - 1 else '')
            ax.set_ylabel('Frequency (Hz)')
    
    plt.show()

## Perform measurement

In [ ]:
# Mesurement configuration
out_file_name = 'data/recorded_data.mat'
deconv_ir_hp = 20 # in Hz
deconv_ir_len = fs # in samples

# Create output data path if it does not exist
Path(out_file_name).parent.mkdir(parents=True, exist_ok=True)

# Wait before starting the measurement
sleep(1) # in s

# Start playback and recording
print(f'\nRecord sweep ... ', end='')
device.reset()
device.start(
    timed=sweep_duration + 1, # in s
    input=True,
    output=True,
    blocking=True,
)
data_rec = device.get_input_data()
print('Done.')

# Deconvolve data
data_ir = acoustics_hardware.processors.deconvolve(
    input=data_rec[0, :],
    output=data_rec[1:, :],
    fs=fs, # in Hz
    f_low=deconv_ir_hp, # in Hz
    f_high=sweep_lp, # in Hz
    T=deconv_ir_len / fs, # in s
)

# Show recording audio preview
for ch in range(data_rec.shape[0]):
    display.display(
        display.Markdown(f'<h3>Channel in_{ch_inputs[ch]}</h3>'),
        display.Audio(data_rec[ch, :], rate=fs),
    )

# Plot recorded data
plot_data(data_rec, fs, ch_labels=ch_inputs)

# Plot IR data
display.display(display.Markdown(f'<h2><center>Deconvolved Impulse Responses</h2>'))
plot_data(data_ir, fs, ch_labels=ch_inputs[1:])
plot_data(data_ir, fs, ch_labels=ch_inputs[1:], is_stacked=True)

# Store data
print(f'Save data to {out_file_name} ... ', end='')
sio.savemat(
    out_file_name,
    {
        'signal': data_rec.T,
        'ir': data_ir.T,
        'fs': fs,
        'ch_input': ch_inputs,
        'ch_output': ch_outputs,
    },
)
print('Done.')